<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3 Interlude: Chisel Standard Library
**Prev: [Generators: Collections](3.2_collections.ipynb)**<br>
**Next: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**

## Motivation
Chisel is all about re-use, so it only makes sense to provide a standard library of interfaces (encouraging interoperability of RTL) and generators for commonly-used hardware blocks.

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# The Cheatsheet
The [Chisel3 cheatsheet](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf) contains a summary of all the major hardware construction APIs, including some of the standard library utilities that we'll introduce below.

# Decoupled, a standard Ready-Valid Interface
One of the commonly used interfaces provided by Chisel is `DecoupledIO`, providing a ready-valid interface for transferring data. The idea is that the sink drives the `bits` signal with the data to be transferred and the `valid` signal when there is data to be transferred. The sink drives the `ready` signal when it is ready to accept data, and data is considered transferred when both `ready` and `valid` are asserted on a cycle.

This provides a flow control mechanism in both directions for data transfer, including a backpressure mechanism.

Note: `ready` and `valid` should not be combinationally coupled, otherwise this may result in unsynthesizable combinational loops. `ready` should only be dependent on whether the sink is able to receive data, and `valid` should only be dependent on whether the source has data. Only after the transaction (on the next clock cycle) should the values update.

Any Chisel data can be wrapped in a `DecoupledIO` (used as the `bits` field) as follows:

```scala
val myChiselData = UInt(8.W)
// or any Chisel data type, such as Bool(), SInt(...), or even custom Bundles
val myDecoupled = Decoupled(myChiselData)
```

The above creates a new `DecoupledIO` `Bundle` with fields
- `valid`: Input(Bool)
- `ready`: Input(Bool)
- `bits`: Output(UInt(8.W))

## Queues


## Arbiters


# Misc Function Blocks
## Bitwise Utilities
### PopCount

### Reverse

## OneHot encoding utilities
### UInt to OneHot
### OneHot to UInt

## Muxes
### OneHot Mux

### Priority Mux


## Counter

---
# You're done!

[Return to the top.](#top)